In [1]:
sentences = [
    "king is a strong man", 
    "queen is a wise woman", 
    "boy is a young man", 
    "girl is a young woman", 
    "prince is a young king", 
    "princess is a young queen", 
    "man is strong", 
    "woman is pretty", 
    "prince is a boy will be king", 
    "princess is a girl will be queen"
]

In [2]:
def getUsefulWords(sentence):
    allWords = sentence.split()
    allDistinctWords = set(allWords)
    uselessWords = {"the","is","be","will","a"}
    allDistinctUsefulWords = list(filter(lambda a: a not in uselessWords, allDistinctWords))
    return allDistinctUsefulWords

In [3]:
allSentenceWordLists = list(map(lambda s: getUsefulWords(s) , sentences))
allUniqueWords = set([item for sublist in allSentenceWordLists for item in sublist])
def makeHotEncoding(i):
    arr = [0]*len(allUniqueWords)
    arr[i] = 1
    return arr
hotEncodings = {element: makeHotEncoding(i) for i,element in enumerate(allUniqueWords)} 
print(hotEncodings)

{'boy': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'king': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'woman': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'girl': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'strong': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'princess': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'prince': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'man': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'pretty': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'queen': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'wise': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'young': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}


In [4]:
def getNeighbors(items):
    neighbors = []
    if len(items) == 1:
        return neighbors
    for i,item in enumerate(items):
        if i != 0:
            neighbors.append([items[i],items[i-1]])
        if i != len(items)-1:
            neighbors.append([items[i],items[i+1]])
    return neighbors

In [5]:
allSentenceNeighborLists = list(map(lambda s: getNeighbors(getUsefulWords(s)) , sentences))
allNeighbors = [item for sublist in allSentenceNeighborLists for item in sublist]
print(allNeighbors)
inputs = tf.convert_to_tensor(list(map(lambda s: hotEncodings[s[0]], allNeighbors)),dtype=tf.float32)
outputs = tf.convert_to_tensor(list(map(lambda s:hotEncodings[s[1]], allNeighbors)),dtype=tf.float32)

[['man', 'strong'], ['strong', 'man'], ['strong', 'king'], ['king', 'strong'], ['queen', 'wise'], ['wise', 'queen'], ['wise', 'woman'], ['woman', 'wise'], ['man', 'boy'], ['boy', 'man'], ['boy', 'young'], ['young', 'boy'], ['young', 'woman'], ['woman', 'young'], ['woman', 'girl'], ['girl', 'woman'], ['prince', 'king'], ['king', 'prince'], ['king', 'young'], ['young', 'king'], ['queen', 'princess'], ['princess', 'queen'], ['princess', 'young'], ['young', 'princess'], ['man', 'strong'], ['strong', 'man'], ['woman', 'pretty'], ['pretty', 'woman'], ['king', 'prince'], ['prince', 'king'], ['prince', 'boy'], ['boy', 'prince'], ['girl', 'princess'], ['princess', 'girl'], ['princess', 'queen'], ['queen', 'princess']]


NameError: name 'tf' is not defined

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

vocab_size = len(allUniqueWords)
embedding_dim=2

maxlen = 500
model = keras.Sequential([
  layers.Embedding(vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 2)           24        
_________________________________________________________________
global_average_pooling1d (Gl (None, 2)                 0         
_________________________________________________________________
dense (Dense)                (None, 16)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(
    inputs,
    outputs,
    epochs=20000,
    batch_size=512,
    validation_split=0.2)

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
lookup = list(map(lambda s: {'name':s[0],'x':s[1][0],'y':s[1][1]}, zip(allUniqueWords,weights)))

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
x_min = 0
y_min = 0
x_max = 0
y_max = 0
x_data = []
y_data = []
for e in lookup:
    x_data.append(e['x'])
    y_data.append(e['y'])
    x_min = min(x_min,e['x'])
    y_min = min(y_min,e['y'])
    x_max = max(x_max,e['x'])
    y_max = max(y_max,e['y'])
    ax.annotate(e['name'],(e['x'],e['y']))
x_pad = abs(x_max-x_min)*.1
y_pad = abs(y_max-y_min)*.1
    
plt.xlim(x_min-x_pad,x_max+x_pad)
plt.ylim(y_min-y_pad,y_max+y_pad)
plt.rcParams["figure.figsize"] = (10,10)
plt.scatter(x_data,y_data)